In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [2]:
edges = cg.read_chunk_edges([cg.get_chunk_id(layer=2,x=50,y=100,z=0)])
edges = edges["between"].get_pairs()
nodes = np.unique(edges)
parents = cg.get_roots(nodes)
d = dict(zip(parents, nodes))
print(d)

{864691138338302132: 75582971913961693, 864691137629078617: 75512534450307707, 864691137995403800: 75582903194484819, 864691137995404568: 75582834475009044, 864691138741942725: 75582971913961542, 864691137995404824: 75582903194484963, 864691136334203586: 75582971913961503}


In [3]:
# edges = np.array([[94803535283198313, 94803466563722624]],dtype=np.uint64)
# between
edges = np.array([[94524946524577880, 94595315268752176]],dtype=np.uint64)

print(cg.get_cross_chunk_edges_layer(edges))
print(cg.get_roots(edges[0]))

[8]
[864691139327226349 864691139527061485]


In [4]:
# parents = np.uint64(edges[0][0])
# while parents and parents.size:
#     parents = cg.get_parent(parents)
#     print(parents)

In [5]:
from pychunkedgraph.graph.edits import add_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt

edges = np.array(edges, dtype=np.uint64)
with TimeIt("add_edge"):
    print(add_edges(cg, atomic_edges=edges))

Start add_edge
   Start get_parents edges
   5.0067901611328125e-05

2 166582540562336100 [166512171818156502]
2 166582540562336100 [166512171818156502]
238589352450461093 [166582540562334005 166512171818156502]

2 166652909306513319 [166652840587035216 166723278050689566]
2 166652909306513319 [166652840587035216 166723278050689566]
238730089938816687 [166652840587035216 166723278050689566 166652909306511714]
Cannot find root id 238589352450461093, 7, 2020-02-13 20:59:02.005000+00:00

3 238730089938816687 [238870827427170695]
3 238730089938816687 [238870827427170695]
310826448204086209 [238730089938813250 238870827427170695]

4 310826448204086209 [311107923180792595]
4 310826448204086209 [311107923180792595]
382959934314213207 [311107923180792595 310826448204081777]

5 382959934314213207 [383522884267609136]
5 382959934314213207 [383522884267609136]
455162629527435266 [383522884267609136 382959934314207198]
Cannot find root id 455162629527435266, 7, 2020-02-13 20:59:06.351000+00:00
Can

In [7]:
cg.get_parents(np.array([595601050722147344, 595601050720271701], dtype=np.uint64))

array([864691128455135245, 668784544668828344], dtype=uint64)

In [10]:
595601050720271701 in cg.node_cache

False

In [14]:
cg.get_parent(600104650349470291)

668784544669702322